In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
from runner.svg2plan import run_new_layout
from new_solutions.selection import *
from domains.domain import Domain

import plotly.io as pio
pio.renderers.default = "browser"

from fixes.problem_types.side_hole_id import *
from visuals.plots import make_subplot_for_all_probs
from export.saver import read_pickle, save_pickle
from helpers.shapely import shape_to_domain
from actions.interfaces import CurrentDomains
from fixes.problem_types.overlap_id import *
from fixes.problem_types.hole_id import *
from actions.actions import create_node_operations

In [15]:
report = run_new_layout("amber_b_f01.svg")
c = Cook(report)


No reference for svg to meters conversion!! 
executer> 
 begining to execute stacking
initializing.. 0
running again -> 1
problem being studied: -> Problem(ix=3, OVERLAP, ['transit_2', 'dining'], resolved=False,  n_actions=2
next best layout transit_2-SQUEEZE-for-OVERLAP near ['transit_2', 'dining']2-PS:0.35-S:1.17


In [16]:
c.run_again()

running again -> 2
problem being studied: -> Problem(ix=1, OVERLAP, ['stair_bottom', 'kitchen'], resolved=False,  n_actions=4
next best layout stair_bottom-SQUEEZE-for-OVERLAP near ['stair_bottom', 'kitchen']2-PS:0.35-S:1.17


In [17]:
c.run_again()

running again -> 3
problem being studied: -> Problem(ix=3, SIDE_HOLE, ['transit_2', 'stair_bottom'], resolved=False,  n_actions=2
next best layout transit_2-STRETCH-for-SIDE_HOLE near ['transit_2', 'stair_bottom']1-PS:0.27-S:0.64


In [18]:
c.run_again()

running again -> 4
problem being studied: -> Problem(ix=2, HOLE, ['wic', 'transit_1', 'm_bath', 'm_bedroom'], resolved=False,  n_actions=4
next best layout transit_1-STRETCH-for-HOLE near ['wic', 'transit_1']1-PS:0.46-S:0.73


In [19]:
c.run_again()

running again -> 5
problem being studied: -> Problem(ix=2, OVERLAP, ['transit_1', 'm_bath'], resolved=False,  n_actions=2
next best layout m_bath-PUSH-for-OVERLAP near ['transit_1', 'm_bath']0-PS:0.24-S:0.12


In [11]:
c.run_again()

running again -> 6
problem being studied: -> Problem(ix=3, OVERLAP, ['bath', 'transit_space'], resolved=False,  n_actions=4
next best layout bath-SQUEEZE-for-OVERLAP near ['bath', 'transit_space']1-PS:0.25-S:0.62


In [12]:
c.run_again()

running again -> 7
problem being studied: -> Problem(ix=2, HOLE, ['transit_space', 'laundry', 'm_bath', 'bath'], resolved=False,  n_actions=4
skipping bc prev domains are in history
next best layout laundry-STRETCH-for-HOLE near ['transit_space', 'laundry']1-PS:0.35-S:0.68


([node: bath, action: STRETCH, summary: Counter({'OVERLAP': 1}), # unres probs: 1,
  node: laundry, action: STRETCH, summary: Counter({'OVERLAP': 1}), # unres probs: 1,
  node: transit_space, action: STRETCH, summary: Counter({'OVERLAP': 1}), # unres probs: 1,
  node: m_bath, action: PULL, summary: Counter({'OVERLAP': 1}), # unres probs: 1,
  node: m_bath, action: STRETCH, summary: Counter({'OVERLAP': 1}), # unres probs: 1,
  node: bath, action: PULL, summary: Counter({'OVERLAP': 1, 'HOLE': 1}), # unres probs: 2,
  node: laundry, action: PULL, summary: Counter({'OVERLAP': 1, 'SIDE_HOLE': 1}), # unres probs: 2,
  node: transit_space, action: PULL, summary: Counter({'OVERLAP': 1, 'HOLE': 1}), # unres probs: 2],
 node: laundry, action: STRETCH, summary: Counter({'OVERLAP': 1}), # unres probs: 1)

In [13]:
c.run_again()

running again -> 8
problem being studied: -> Problem(ix=2, OVERLAP, ['m_bath', 'laundry'], resolved=False,  n_actions=2
next best layout m_bath-SQUEEZE-for-OVERLAP near ['m_bath', 'laundry']0-PS:0.26-S:0.13


In [17]:
c.res_hist[7], c.bl_hist[7]

([node: m_bath, action: SQUEEZE, summary: Counter(), # unres probs: 0,
  node: laundry, action: SQUEEZE, summary: Counter({'HOLE': 1}), # unres probs: 1,
  node: m_bath, action: PUSH, summary: Counter({'OVERLAP': 1}), # unres probs: 1,
  node: laundry, action: PUSH, summary: Counter({'OVERLAP': 1, 'HOLE': 1}), # unres probs: 2],
 node: m_bath, action: PUSH, summary: Counter({'OVERLAP': 1}), # unres probs: 1)

In [18]:
[i.short_message() for i in c.res_hist[7]]

["m_bath-SQUEEZE-for-OVERLAP near ['m_bath', 'laundry']0-PS:0.26-S:0.13",
 "laundry-SQUEEZE-for-OVERLAP near ['m_bath', 'laundry']1-PS:0.35-S:0.68",
 "m_bath-PUSH-for-OVERLAP near ['m_bath', 'laundry']1-PS:0.52-S:0.76",
 "laundry-PUSH-for-OVERLAP near ['m_bath', 'laundry']2-PS:0.70-S:1.35"]

In [31]:
c.run_again()

problem being studied: -> Problem(ix=2, OVERLAP, ['m_bath', 'm_bedroom'], resolved=False,  n_actions=2
next best layout m_bedroom-PUSH-for-OVERLAP near ['m_bath', 'm_bedroom']1-PS:0.51-S:0.75
running again -> 9


In [12]:
c.run_again()

problem being studied: -> Problem(ix=2, HOLE, ['bath', 'dining', 'm_bedroom', 'm_bath'], resolved=False,  n_actions=4
next best layout m_bath-PULL-for-HOLE near ['bath', 'dining']0-PS:0.25-S:0.12
running again -> 10


In [20]:
c.plot_all()

len sols = 5. Len indices = 9
